## Task 2: Basic ML on MNIST and FashionMNIST

In [1]:
using MLDatasets, Plots, Measures, LaTeXStrings, StatsBase, Random, LinearAlgebra
include("src/ML.jl")

logistic_one_vs_all_auto_diff (generic function with 1 method)

### Task 2.1 One vs. all (rest) Linear and Logistic

#### Linear Model one vs all
The linear model on both the standard MINST and the fashionMINST

In [2]:
train_data_MINST = MLDatasets.MNIST.traindata(Float64);
test_data_MINST = MLDatasets.MNIST.testdata(Float64);

ConfusionMatrix_MINST, acc_MINST = linear_one_vs_all(train_data_MINST, test_data_MINST, 9);


In [3]:
println("Accuracy: ", acc_MINST)
println("ConfusionMatrix: ")
show(stdout, "text/plain", ConfusionMatrix_MINST)

Accuracy: 0.8603
ConfusionMatrix: 
10×10 Matrix{Int64}:
 944     0   18    4    0   23   18    5   14   15
   0  1107   54   17   22   18   10   40   46   11
   1     2  813   23    6    3    9   16   11    2
   2     2   26  880    1   72    0    6   30   17
   2     3   15    5  881   24   22   26   27   80
   7     1    0   17    5  659   17    0   40    1
  14     5   42    9   10   23  875    1   15    1
   2     1   22   21    2   14    0  884   12   77
   7    14   37   22   11   39    7    0  759    4
   1     0    5   12   44   17    0   50   20  801

In [4]:
train_data_fMINST = FashionMNIST.traindata(Float64);
test_data_fMINST = FashionMNIST.testdata(Float64);

ConfusionMatrix_fMINST, acc_fMINST = linear_one_vs_all(train_data_fMINST, test_data_fMINST, 9);

In [5]:
println("Accuracy: ", acc_fMINST)
println("ConfusionMatrix: ")
show(stdout, "text/plain", ConfusionMatrix_fMINST)

Accuracy: 0.8113
ConfusionMatrix: 
10×10 Matrix{Int64}:
 802    4   38   22    1    0  160    0    5    0
   3  952    1   13    6    0    3    0    0    0
  12    5  690   14  114    0  136    0    4    0
  71   29   16  852   37    2   57    0   12    0
   9    5  169   47  750    0  118    0    6    0
  19    2   15   23    7  838   21   32   22   22
  62    2   56   21   77    2  462    0   11    1
   1    0    0    0    0   88    1  909    7   48
  21    1   15    8    8   14   42    0  929    0
   0    0    0    0    0   56    0   59    4  929

The values above show that for the standard MINST data set had a accuracy of 0.8603 while the fashion MINST data set had an accuracy of 0.8113 for the linear machine learning model. The confucsion matrix for each model give an indicaiton of where the most mistakes were made. Which was row 5 column 10 for the standard MINST which indicates that a four being mistaken as a nine is the most common error. While for the fashion MINST was row 5 column 3 which indicates that a coat being mistaken as a pullover is the most common error for the fashion dataset.

#### Logistic Model one vs all

While trying to implement the one vs all logistic model I had great difficulties in getting the explicit differientation to work. Below show my attempt at a solution however the code was very slow and fails to do even one epoch in a reasonable amount of time. 

In [6]:
train_data_MINST = MLDatasets.MNIST.traindata(Float64);
test_data_MINST = MLDatasets.MNIST.testdata(Float64);
logistic_one_vs_all(train_data_MINST, test_data_MINST);

Batch Number = 1 (0.91 sec)
Batch Number = 2 (28.36 sec)
Batch Number = 3 (28.11 sec)
Batch Number = 4 (28.49 sec)
Batch Number = 5 (28.0 sec)
Total elpased time after 5 batches 113.86


I computed five 1000 length batches for this example and this took my computer 2 mintues. To compute one epoch at a rate of 30 seconds per batch would take 30 minutes (Assuming the computation time doesnt increase) which is obviously not right and definitly not feasible for multiple epochs. I've include this because an attempt is better than nothing but this would be useless in providing an accurate guess of any image.

### Task 2.2 One vs. One Linear and Logistic

#### Linear one vs one
Since as noted above I failed to devolp a logistic one vs all model for one vs one only the linear model will be tested. 

MINST Dataset

In [7]:
#Collecting data
train_data_MINST = MLDatasets.MNIST.traindata(Float64);
test_data_MINST = MLDatasets.MNIST.testdata(Float64);
train_imgs = train_data_MINST[1]
train_labels = train_data_MINST[2]

test_imgs = test_data_MINST[1]
test_labels = test_data_MINST[2]

#sorting data
X_sorted = vcat([vec(train_imgs[:,:,k])' for k in sortperm(train_labels)]...);
Xt_sorted = vcat([vec(test_imgs[:,:,k])' for k in sortperm(test_labels)]...);


In [8]:
#organising data
Intervals = []

for i in 1:10
    push!(Intervals, (counts(train_labels)[i], counts(test_labels)[i]))
end

X_values = []

start = [1, 1]
finish = [counts(train_labels)[1], counts(test_labels)[1]]
for i in 1:10
    push!(X_values, (X_sorted[start[1]:finish[1],:], Xt_sorted[start[2]:finish[2],:]))

    i == 10 && break #break when i == 10 to avoid index out of bounds error
    start[1] = start[1] + counts(train_labels)[i]
    start[2] = start[2] + counts(test_labels)[i]
    finish[1] = finish[1] + counts(train_labels)[i+1]
    finish[2] = finish[2] + counts(test_labels)[i+1]
end

In [9]:
#Recording accuracy values for 45 linear models
acc = 0
num_of_tests = 9
counter = 0

#testing all permutation without repretition
for i in 1:10
    
    for j in 11-num_of_tests:10
        acc += linear_one_vs_one(X_values[i], X_values[j], Intervals[i], Intervals[j])
        counter += 1
    end
    num_of_tests -= 1
    
end

println("The average accuraccy of the linear one vs one model is ", acc/45)

The average accuraccy of the linear one vs one model is 0.983040946104759


FashionMINST Dataset

In [10]:
#collecting data 
train_data_fMINST = FashionMNIST.traindata(Float64);
test_data_fMINST = FashionMNIST.testdata(Float64)

train_imgs = train_data_fMINST[1]
train_labels = train_data_fMINST[2]

test_imgs = test_data_fMINST[1]
test_labels = test_data_fMINST[2]

#values in the FashionMNIST are naturally sorted
X_sorted = vcat([vec(train_imgs[:,:,k])' for k in 1:length(train_labels)]...);
Xt_sorted = vcat([vec(test_imgs[:,:,k])' for k in 1:length(test_labels)]...);

In [11]:
#organising data
Intervals = []

for i in 1:10
    push!(Intervals, (counts(train_labels)[i], counts(test_labels)[i]))
end

X_values = []

start = [1, 1]
finish = [counts(train_labels)[1], counts(test_labels)[1]]
for i in 1:10
    push!(X_values, (X_sorted[start[1]:finish[1],:], Xt_sorted[start[2]:finish[2],:]))

    i == 10 && break #break when i == 10 to avoid index out of bounds error
    start[1] = start[1] + counts(train_labels)[i]
    start[2] = start[2] + counts(test_labels)[i]
    finish[1] = finish[1] + counts(train_labels)[i+1]
    finish[2] = finish[2] + counts(test_labels)[i+1]
end

In [12]:
#Recording accuracu values for 45 linear models
acc = 0
num_of_tests = 9
counter = 0

#testing all permutation without repretition
for i in 1:10
    
    for j in 11-num_of_tests:10
        acc += linear_one_vs_one(X_values[i], X_values[j], Intervals[i], Intervals[j])
        counter += 1
    end
    num_of_tests -= 1
    
end

println("The average accuraccy of the linear one vs one model is ", acc/45)

The average accuraccy of the linear one vs one model is 0.5024


The one vs one linear ML model for the MINST data set had a very high accuracy value of 0.98 compared to the significantly lower value for the FashionMINST dada set which had an average accuracy value of 0.50. I was expecting a higher accuracy value for the the MINST data set because each digit has very distinct features while many of the Fashion type overlap significantly which would have lowered the overall accuracy value.

### Task 2.3 Multi-class classifier (logistic softmax)

### Task 2.4 Comparison of results and discussion